# 天池大赛·印象盐城

## 时间分析

初赛提供2012年1月-2017年10月盐城分车型销量配置数据。
第一阶段需要参赛者预测2017年11月盐城分车型销量数据，第二阶段需要参赛者预测2017年12月盐城分车型销量数据。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../[new] yancheng_train_20171226.csv', low_memory=False)
test = pd.read_csv('../yancheng_testA_20171225.csv')
trainFeature = pd.read_csv('train_feature.csv')

In [3]:
trainDate = train[['sale_date', 'sale_quantity', 'class_id']]
trainDate = trainDate.groupby(['sale_date', 'class_id']).agg('sum').reset_index()
trainDate['year'] = trainDate['sale_date'].apply(lambda x: int(str(x)[0:4]))
trainDate['mouth'] = trainDate['sale_date'].apply(lambda x: int(str(x)[4:6]))
trainDate = trainDate.drop(['sale_date'], axis=1)
trainDate

,class_id,sale_quantity,year,mouth
0,125403,90,2012,1
1,136916,665,2012,1
2,178529,1739,2012,1
3,194450,65,2012,1
4,198427,47,2012,1
5,209945,168,2012,1
6,248352,1713,2012,1
7,281301,965,2012,1
8,290854,632,2012,1
9,291086,507,2012,1


### 提取时间信息

In [4]:
trainSaleDate = trainFeature

# 2012年1月的销量信息
trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['mouth']==1)]
# trainTmp = trainTmp[trainTmp['mouth']==1]
trainTmp.loc[:, 'how_many_mouth_has_sell'] = 1
sale_class_count = len(trainTmp)

trainSaleDate = pd.merge(trainSaleDate, trainTmp, on='class_id', how='left')
trainSaleDate['year'].fillna(2012.0, inplace=True)
trainSaleDate['mouth'].fillna(1.0, inplace=True)
# trainSaleDate['how_many_mouth_has_sell'].fillna(0.0, inplace=True)
trainSaleDate.loc[:, 'sale_quantity_total_last_mouth'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_mouth'] = 0
trainSaleDate.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
# trainSaleDate


for mouth in range(2, 13):
    trainSaleDateTmp = trainFeature
    trainTmp = trainDate[(trainDate['year']==2012)&(trainDate['mouth']==mouth)]
    
    sellMouth = trainSaleDate[['class_id', 'how_many_mouth_has_sell', 'sale_quantity']][(trainSaleDate['year']==2012)&(trainSaleDate['mouth']==(mouth-1))]
    sellMouth.dropna(inplace=True)
    sellMouth['how_many_mouth_has_sell'] = sellMouth['how_many_mouth_has_sell']+1
    sale_quantity_total_last_mouth = sellMouth['sale_quantity'].sum()
    sellMouth.rename(columns={'sale_quantity': 'sale_quantity_this_last_mouth'}, inplace=True)
    
    trainTmp = pd.merge(trainTmp, sellMouth, on='class_id', how='left')
    trainTmp['how_many_mouth_has_sell'].fillna(1.0, inplace=True)
    sale_class_count = len(trainTmp)
    
    trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
    trainSaleDateTmp['year'].fillna(2012.0, inplace=True)
    trainSaleDateTmp['mouth'].fillna(mouth, inplace=True)
    trainSaleDateTmp.loc[:, 'sale_quantity_total_last_mouth'] = sale_quantity_total_last_mouth
    trainSaleDateTmp.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
    
    trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
    
trainSaleDate.loc[:, 'sale_quantity_total_last_year'] = 0
trainSaleDate.loc[:, 'sale_quantity_this_last_year'] = 0

for year in range(2013, 2017):
    for mouth in range(1, 13):
        trainSaleDateTmp = trainFeature
        trainTmp = trainDate[(trainDate['year']==year)&(trainDate['mouth']==mouth)]
        
        if mouth == 1:
            sellMouth = trainSaleDate[['class_id', 'how_many_mouth_has_sell', 'sale_quantity']][(trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']==12)]
        else:
            sellMouth = trainSaleDate[['class_id', 'how_many_mouth_has_sell', 'sale_quantity']][(trainSaleDate['year']==year)&(trainSaleDate['mouth']==(mouth-1))]
        sellMouth.dropna(inplace=True)
        sellMouth['how_many_mouth_has_sell'] = sellMouth['how_many_mouth_has_sell']+1
        sale_quantity_total_last_mouth = sellMouth['sale_quantity'].sum()
        sellMouth.rename(columns={'sale_quantity': 'sale_quantity_this_last_mouth'}, inplace=True)
        
        sellYear = trainSaleDate[['class_id', 'sale_quantity']][((trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']>=mouth))|((trainSaleDate['year']==year)&(trainSaleDate['mouth']<mouth))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainTmp = pd.merge(trainTmp, sellMouth, on='class_id', how='left')
        trainTmp['how_many_mouth_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainTmp)
        
        trainTmp = pd.merge(trainTmp, sellYear, on='class_id', how='left')
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
        trainSaleDateTmp['year'].fillna(year, inplace=True)
        trainSaleDateTmp['mouth'].fillna(mouth, inplace=True)
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_mouth'] = sale_quantity_total_last_mouth
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])
        
for year in range(2017, 2018):
    for mouth in range(1, 11):
        trainSaleDateTmp = trainFeature
        trainTmp = trainDate[trainDate['year']==year]
        trainTmp = trainTmp[trainTmp['mouth']==mouth]
        
        if mouth == 1:
            sellMouth = trainSaleDate[['class_id', 'how_many_mouth_has_sell', 'sale_quantity']][(trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']==12)]
        else:
            sellMouth = trainSaleDate[['class_id', 'how_many_mouth_has_sell', 'sale_quantity']][(trainSaleDate['year']==year)&(trainSaleDate['mouth']==(mouth-1))]
        sellMouth.dropna(inplace=True)
        sellMouth['how_many_mouth_has_sell'] = sellMouth['how_many_mouth_has_sell']+1
        sale_quantity_total_last_mouth = sellMouth['sale_quantity'].sum()
        sellMouth.rename(columns={'sale_quantity': 'sale_quantity_this_last_mouth'}, inplace=True)
        
        sellYear = trainSaleDate[['class_id', 'sale_quantity']][((trainSaleDate['year']==(year-1))&(trainSaleDate['mouth']>=mouth))|((trainSaleDate['year']==year)&(trainSaleDate['mouth']<mouth))]
        sellYear.dropna(inplace=True)
        sellYear = sellYear.groupby('class_id').agg('sum').reset_index()
        sale_quantity_total_last_year = sellYear['sale_quantity'].sum()
        sellYear.rename(columns={'sale_quantity': 'sale_quantity_this_last_year'}, inplace=True)
        
        trainTmp = pd.merge(trainTmp, sellMouth, on='class_id', how='left')
        trainTmp['how_many_mouth_has_sell'].fillna(1.0, inplace=True)
        sale_class_count = len(trainTmp)
        
        trainTmp = pd.merge(trainTmp, sellYear, on='class_id', how='left')
        
        trainSaleDateTmp = pd.merge(trainSaleDateTmp, trainTmp, on='class_id', how='left')
        trainSaleDateTmp['year'].fillna(year, inplace=True)
        trainSaleDateTmp['mouth'].fillna(mouth, inplace=True)
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_mouth'] = sale_quantity_total_last_mouth
        trainSaleDateTmp.loc[:, 'sale_quantity_total_last_year'] = sale_quantity_total_last_year
        trainSaleDateTmp.loc[:, 'this_mouth_sale_class_count'] = sale_class_count
        
        trainSaleDate = pd.concat([trainSaleDate, trainSaleDateTmp])

trainSaleDate.sort(['year', 'mouth', 'class_id'])
trainSaleDate.fillna(0.0, inplace=True)
trainSaleDate.to_csv('trainSaleDate.csv', index=False)

C:\Users\yudake\Anaconda3\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\yudake\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:115: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
